In [ ]:
%cd ../

from src.requirements import *
from src.ppca import PPCA
from src.utils import *
%matplotlib inline

## ******************************************************************************************************************************
## OPEN CONFIG FILE
## ******************************************************************************************************************************

with open('configFile.json') as json_data_file:
    configFile = json.load(json_data_file)

## ******************************************************************************************************************************
## CARTO
## ******************************************************************************************************************************

carto_username = configFile['config']['CARTO']['username']
carto_API = configFile['config']['CARTO']['API_key']

creds = Credentials(carto_username, carto_API)
set_default_credentials(creds)

%cd models/

# Upload data to CARTO

## Read States geometries and merge with Counties geometries

In [ ]:
gdf_states = gpd.read_file('../data/cb_2018_us_state_500k.shp')
gdf_states.to_crs(epsg=4326, inplace = True)
to_carto(gdf_states,"us_states_geoms", if_exists = 'replace')

## Lockdown phase

In [ ]:
phase = 'drop'
admunit = 'county'
pc_var_thr = '0.55'
filename = 'mobility_google_county_{}'.format(phase)
df_drop = pd.read_csv('../data/{}_{}_random.csv'.format(filename,pc_var_thr))

## Read Counties geometries
gdf = gpd.read_file("../data/{}.shp".format(filename))
gdf['geoid_state'] = gdf['geoid'].apply(lambda x: x[:2])

### The dataframe has the number of rows equal to 2 times the number of selected counties (1946 * 2) where the first 1946 rows correspond to the sum of the "iid"+"besag" effects

In [ ]:
df_drop = df_drop.iloc[:int(df_drop.shape[0]/2),:]

### Merge data and geometries and upload

In [ ]:
gdf_drop = gdf.merge(df_drop, on = 'ID')
gdf_drop['exp_mean'] = np.exp(gdf_drop['mean'])
to_carto(gdf_drop,"{}_random_sp".format(filename), if_exists = 'replace')

## Recovery phase

In [ ]:
phase = 'up'
filename = 'mobility_google_county_{}'.format(phase)
df_up = pd.read_csv('../data/{}_{}_random.csv'.format(filename,pc_var_thr))

### The dataframe has the number of rows equal to 2 times the number of selected counties (1946 * 2) where the first 1946 rows correspond to the sum of the "iid"+"besag" effects

In [ ]:
df_up = df_up.iloc[:int(df_up.shape[0]/2),:]

### Merge data and geometries and upload

In [ ]:
gdf_up = gdf.merge(df_up, on = 'ID')
gdf_up['exp_mean'] = np.exp(gdf_up['mean'])
to_carto(gdf_up,"{}_random_sp".format(filename), if_exists = 'replace')

# Make a map of the posterior mean of the BYM random effect

In [ ]:
pubmap_random_drop = Map([Layer("mobility_google_county_drop_random_sp",
                             encode_data=False,
                             style = color_bins_style('mean', 
                                               stroke_width= 0.5,
                                               palette = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020'],
                                               breaks = [-1,-0.75,-0.5,-0.25,0,0.25,0.5,0.75,1],                                                           
                                               opacity = 1),
                             legends = color_bins_legend(title='Lockdown phase', 
                                   description='Posterior mean of the spatial random effects (corresponding to the sum of the "IID" and "ICAR" terms)', 
                                   footer =''),
                             popup_hover=[popup_element('mean', title='Spatial Random Effects')]),
                             Layer("us_states_geoms", 
                                   style = basic_style(color = 'white', 
                                                       stroke_width= 2.5,
                                                       opacity = 0),
                                    popup_hover=[popup_element('name', title='State')])],
                                                viewport={'zoom': 2, 'lat': 40.587755, 'lng': -102.973985}
)

In [ ]:
pubmap_random_up = Map([Layer("mobility_google_county_up_random_sp",
                             encode_data=False,
                             style = color_bins_style('mean', 
                                               stroke_width= 0.5,
                                               palette = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020'],
                                               breaks = [-1,-0.75,-0.5,-0.25,0,0.25,0.5,0.75,1],                                                           
                                               opacity = 1),
                             legends = color_bins_legend(title='Recovery phase', 
                                   description='Posterior mean of the spatial random effects (corresponding to the sum of the "IID" and "ICAR" terms)', 
                                   footer =''),
                             popup_hover=[popup_element('mean', title='Spatial Random Effects')]),
                             Layer("us_states_geoms", 
                                   style = basic_style(color = 'white', 
                                                       stroke_width= 2.5,
                                                       opacity = 0),
                                    popup_hover=[popup_element('name', title='State')])]
)

In [ ]:
from cartoframes.viz import Layout
pubmap = Layout([pubmap_random_drop, pubmap_random_up],
                1,2,
                is_static=False,map_height=500, 
                viewport={'zoom': 2.7, 'lat': 40.587755, 'lng': -102.973985}
)

In [ ]:
pubmap.publish("mobility_{}_random_spatial".format(admunit), 
               password=None,
               if_exists='replace')